Demonstrations for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.linear_algebra.tensors</a>
========

In [1]:
import proveit
from proveit import defaults
from proveit import Function, ExprRange, ExprTuple, IndexedVar
from proveit import a, f, i, u, v, w, x, y, z, alpha, fi
from proveit.logic import Forall, Equals, NotEquals, InSet, CartExp
from proveit.numbers import Natural, Real, Complex
from proveit.numbers import one, two, three, four, five, Interval
from proveit.linear_algebra import (
    VecSpaces, VecAdd, VecSum, VecZero, ScalarMult, TensorProd, TensorExp)
%begin demonstrations

### Vector space default assumptions

In order to apply the tensor product theorems, we need the operands to be known as vectors in vector spaces over a common field.  For convenience, we may specify a default field, or we may specify a field when calling the TensorProd methods.

Let's set some defaults for convienience in our testing below.

In [2]:
R3 = CartExp(Real, three)

R3: Real^{3}

In [3]:
C3 = CartExp(Complex, three)

C3: Complex^{3}

In [4]:
defaults.assumptions = [
    InSet(u, R3), InSet(v, R3), InSet(w, R3), 
    InSet(x, R3), InSet(y, R3), InSet(z, R3),
    NotEquals(x, VecZero(R3)), NotEquals(z, VecZero(R3)),
    InSet(a, Real), InSet(alpha, Complex), 
    Forall(i, InSet(fi, R3), domain=Natural)]

defaults.assumptions: (u in Real^{3}, v in Real^{3}, w in Real^{3}, x in Real^{3}, y in Real^{3}, z in Real^{3}, x != 0(Real^{3}), z != 0(Real^{3}), a in Real, alpha in Complex, forall_{i in Natural} (f(i) in Real^{3}))

In [5]:
summand_assumption = defaults.assumptions[-1]

summand_assumption: forall_{i in Natural} (f(i) in Real^{3})

In [6]:
summand_assumption.instantiate(assumptions=[summand_assumption,
                                            InSet(i, Natural)])

{forall_{i in Natural} (f(i) in Real^{3}), i in Natural} |- f(i) in Real^{3}

### Some Example `TensorProd` For Testing

In [7]:
tensor_prod_00 = TensorProd(ScalarMult(a, x), y)

tensor_prod_00: (a * x) otimes y

In [8]:
tensor_prod_000 = TensorProd(x, ScalarMult(a, y))

tensor_prod_000: x otimes (a * y)

In [9]:
tensor_prod_01 = TensorProd(ScalarMult(a, x), y, z)

tensor_prod_01: (a * x) otimes y otimes z

In [10]:
tensor_prod_02 = TensorProd(x, ScalarMult(a, y), z)

tensor_prod_02: x otimes (a * y) otimes z

In [11]:
tensor_prod_03 = TensorProd(x, y, ScalarMult(a, z))

tensor_prod_03: x otimes y otimes (a * z)

In [12]:
tensor_prod_04 = TensorProd(u, TensorProd(v, ScalarMult(a, w), x, 
                                          ScalarMult(alpha, y)))

tensor_prod_04: u otimes (v otimes (a * w) otimes x otimes (alpha * y))

In [13]:
tensor_prod_05 = TensorProd(u, ScalarMult(a, v), VecAdd(w, x, y), z)

tensor_prod_05: u otimes (a * v) otimes (w + x + y) otimes z

In [14]:
tensor_prod_06 = TensorProd(VecAdd(x, y), z)

tensor_prod_06: (x + y) otimes z

In [15]:
tensor_prod_07 = TensorProd(x, VecAdd(y, z))

tensor_prod_07: x otimes (y + z)

In [16]:
tensor_prod_08 = TensorProd(u, v, VecAdd(w, x, y), z)

tensor_prod_08: u otimes v otimes (w + x + y) otimes z

In [17]:
tensor_prod_with_sum_01 = TensorProd(x, VecSum(i, Function(f, i), domain=Interval(one, three)), z)

tensor_prod_with_sum_01: x otimes (Sum_{i=1}^{3} f(i)) otimes z

In [18]:
tensor_prod_with_sum_02 = TensorProd(VecSum(i, Function(f, i), domain=Interval(two, four)), z)

tensor_prod_with_sum_02: (Sum_{i=2}^{4} f(i)) otimes z

In [19]:
tensor_prod_with_sum_03 = TensorProd(x, VecSum(i, Function(f, i), domain=Interval(two, four)))

tensor_prod_with_sum_03: x otimes (Sum_{i=2}^{4} f(i))

In [20]:
tensor_prod_with_sum_04 = TensorProd(u, v, w, x, VecSum(i, Function(f, i), domain=Interval(one, five)), z)

tensor_prod_with_sum_04: u otimes v otimes w otimes x otimes (Sum_{i=1}^{5} f(i)) otimes z

Upon implementing <a href=https://github.com/PyProveIt/Prove-It/issues/28>Issue #28</a>, or something along those lines, the following should not be necessary:

In [21]:
i_domains = [tensor_prod_with_sum_01.operands[1].domain,
             tensor_prod_with_sum_03.operands[1].domain,
             tensor_prod_with_sum_04.operands[-2].domain]
judgments = []
for i_domain in i_domains:
    judgment = summand_assumption.instantiate(
        assumptions=[summand_assumption, InSet(i, i_domain)])
    judgments.append(judgment)
judgments

### `TensorProd` simplification

In [22]:
help(TensorProd.shallow_simplification)

Help on function shallow_simplification in module proveit.linear_algebra.tensors.tensor_prod:

shallow_simplification(self, *, must_evaluate=False, **defaults_config)
    Returns a proven simplification equation for this TensorProd
    expression assuming the operands have been simplified.
    
    Currently deals only with:
    (1) simplifying a TensorProd(x) (i.e. a TensorProd with a
        single operand x) to x itself. For example,
        TensorProd(x) = x.
    (2) Ungrouping nested tensor products.
    (3) Factoring out scalars.
    
    Keyword arguments are accepted for temporarily changing any
    of the attributes of proveit.defaults.
    
    'shallow_simplified' returns the right-hand side of 'shallow_simplification'.
    'shallow_simplify', called on an InnerExpr of a Judgment,
    substitutes the right-hand side of 'shallow_simplification' for
    the inner expression.



First test out unary `TensorProd` simplification.

In [23]:
TensorProd(x).simplification()

{x in Real^{3}} |- [otimes](x) = x

Our next test will involve ungrouping and pulling out scalar factors but will not work without a proper default field specified since it mixes real vectors with a complex scalar.

In [24]:
tensor_prod_04

u otimes (v otimes (a * w) otimes x otimes (alpha * y))

In [25]:
try:
    tensor_prod_04.simplification()
    assert False, "Expecting an error"
except ValueError as e:
    print("Expected error: %s"%e)

Expected error: A field for vector spaces was not specified and VecSpaces.default_field was not set.


In [26]:
VecSpaces.default_field = Real

VecSpaces.default_field: Real

In [27]:
from proveit import InstantiationFailure
try:
    tensor_prod_04.simplification()
    assert False, "Expecting an error"
except InstantiationFailure as e:
    print("Expected error: %s"%e)

Expected error: Proof step failed assuming {z != 0(Real^{3}), x in Real^{3}, y in Real^{3}, v in Real^{3}, alpha in Complex, w in Real^{3}, z in Real^{3}, a in Real, forall_{i in Natural} (f(i) in Real^{3}), u in Real^{3}, x != 0(Real^{3})}:
Attempting to instantiate |- forall_{K} [forall_{V in_c VecSpaces(K)} [forall_{a in K} [forall_{x in V} ((a * x) in V)]]] with {K: Real, V: Real^{3}, a: alpha, x: y}:
Unsatisfied condition: alpha in Real


In [28]:
VecSpaces.default_field = Complex

VecSpaces.default_field: Complex

In [29]:
tensor_prod_04.simplification()

{x in Real^{3}, y in Real^{3}, v in Real^{3}, alpha in Complex, w in Real^{3}, a in Real, u in Real^{3}} |- (u otimes (v otimes (a * w) otimes x otimes (alpha * y))) = ((a * alpha) * (u otimes v otimes w otimes x otimes y))

### The `TensorProd.association()` and `TensorProd.disassociation()` methods

In [30]:
help(TensorProd.association)

Help on function association in module proveit.linear_algebra.tensors.tensor_prod:

association(self, start_idx, length, *, field=None, **defaults_config)
    Given vector operands, deduce that this expression is equal 
    to a form in which operands in the
    range [start_idx, start_idx+length) are grouped together.
    For example, (a ⊗ b ⊗ ... ⊗ y ⊗ z) = 
        (a ⊗ b ... ⊗ (l ⊗ ... ⊗ m) ⊗ ... ⊗ y ⊗ z)
    
    For this to work, the operands must be known to be in
    vector spaces of a common field.  If the field is not specified,
    then VecSpaces.default_field is used.
    
    Keyword arguments are accepted for temporarily changing any
    of the attributes of proveit.defaults.
    
    'associated' returns the right-hand side of 'association'.
    'associate', called on an InnerExpr of a Judgment,
    substitutes the right-hand side of 'association' for
    the inner expression.



In [31]:
help(TensorProd.disassociation)

Help on function disassociation in module proveit.linear_algebra.tensors.tensor_prod:

disassociation(self, idx, *, field=None, **defaults_config)
    Given vector operands, deduce that this expression is equal 
    to a form in which the operand
    at index idx is no longer grouped together.
    For example, (a ⊗ b ... ⊗ (l ⊗ ... ⊗ m) ⊗ ... ⊗ y⊗ z) 
        = (a ⊗ b ⊗ ... ⊗ y ⊗ z)
    
    For this to work, the operands must be known to be in
    vector spaces of a common field.  If the field is not specified,
    then VecSpaces.default_field is used.
    
    Keyword arguments are accepted for temporarily changing any
    of the attributes of proveit.defaults.
    
    'disassociated' returns the right-hand side of 'disassociation'.
    'disassociate', called on an InnerExpr of a Judgment,
    substitutes the right-hand side of 'disassociation' for
    the inner expression.



In [32]:
tensor_prod_with_sum_04.association(1, 4)

{x in Real^{3}, v in Real^{3}, w in Real^{3}, z in Real^{3}, forall_{i in Natural} (f(i) in Real^{3}), u in Real^{3}} |- (u otimes v otimes w otimes x otimes (Sum_{i=1}^{5} f(i)) otimes z) = (u otimes (v otimes w otimes x otimes (Sum_{i=1}^{5} f(i))) otimes z)

In [33]:
tensor_prod_04.disassociation(1, auto_simplify=False)

{x in Real^{3}, y in Real^{3}, v in Real^{3}, alpha in Complex, w in Real^{3}, a in Real, u in Real^{3}} |- (u otimes (v otimes (a * w) otimes x otimes (alpha * y))) = (u otimes v otimes (a * w) otimes x otimes (alpha * y))

### The `TensorProd.scalar_factorization()` method

In [34]:
help(TensorProd.scalar_factorization)

Help on function scalar_factorization in module proveit.linear_algebra.tensors.tensor_prod:

scalar_factorization(self, idx=None, *, field=None, **defaults_config)
    Prove the factorization of a scalar from one of the tensor 
    product operands and return the original tensor product equal 
    to the factored version.  If idx is provided, it will specify 
    the (0-based) index location of the ScalarMult operand with the
    multiplier to factor out.  If no idx is provided, the first 
    ScalarMult operand will be targeted.
    
    For example,
        TensorProd(a, ScalarMult(c, b), d).factorization(1)
    returns
        |- TensorProd(a, ScalarMult(c, b), d) =
           c TensorProd(a, b, d)
    
    As a prerequisite, the operands must be known to be vectors in
    vector spaces over a common field which contains the scalar
    multiplier being factored.  If the field is not specified,
    then VecSpaces.default_field is used.
    
    Keyword arguments are accepted for temp

In [35]:
tensor_prod_00

(a * x) otimes y

In [36]:
tensor_prod_00.scalar_factorization(0, field=Real)

{x in Real^{3}, y in Real^{3}, a in Real} |- ((a * x) otimes y) = (a * (x otimes y))

By default, the first ScalarMult operand will be the target.  Also, we may use the default field, `VecSpaces.default_field`.

In [37]:
VecSpaces.default_field = Real

VecSpaces.default_field: Real

In [38]:
tensor_prod_000.scalar_factorization()

{x in Real^{3}, y in Real^{3}, a in Real} |- (x otimes (a * y)) = (a * (x otimes y))

In [39]:
tensor_prod_01.scalar_factorization()

{x in Real^{3}, y in Real^{3}, z in Real^{3}, a in Real} |- ((a * x) otimes y otimes z) = (a * (x otimes y otimes z))

In [40]:
tensor_prod_02.scalar_factorization(1)

{x in Real^{3}, y in Real^{3}, z in Real^{3}, a in Real} |- (x otimes (a * y) otimes z) = (a * (x otimes y otimes z))

In [41]:
tensor_prod_03.scalar_factorization()

{x in Real^{3}, y in Real^{3}, z in Real^{3}, a in Real} |- (x otimes y otimes (a * z)) = (a * (x otimes y otimes z))

In [42]:
tensor_prod_05.scalar_factorization()

{x in Real^{3}, y in Real^{3}, v in Real^{3}, w in Real^{3}, z in Real^{3}, a in Real, u in Real^{3}} |- (u otimes (a * v) otimes (w + x + y) otimes z) = (a * (u otimes v otimes (w + x + y) otimes z))

### The `TensorProd.distribution()` method

In [43]:
help(TensorProd.distribution)

Help on function distribution in module proveit.linear_algebra.tensors.tensor_prod:

distribution(self, idx, *, field=None, **defaults_config)
    Given a TensorProd operand at the (0-based) index location
    'idx' that is a vector sum or summation, prove the distribution
    over that TensorProd factor and return an equality to the 
    original TensorProd. For example, we could take the TensorProd
        tens_prod = TensorProd(a, b+c, d)
    and call tens_prod.distribution(1) to obtain:
        |- TensorProd(a, b+c, d) =
           TensorProd(a, b, d) + TensorProd(a, c, d)
    
    Keyword arguments are accepted for temporarily changing any
    of the attributes of proveit.defaults.
    
    'distributed' returns the right-hand side of 'distribution'.
    'distribute', called on an InnerExpr of a Judgment,
    substitutes the right-hand side of 'distribution' for
    the inner expression.



In [44]:
tensor_prod_05

u otimes (a * v) otimes (w + x + y) otimes z

In [45]:
tensor_prod_05.distribution(2)

{x in Real^{3}, y in Real^{3}, v in Real^{3}, w in Real^{3}, z in Real^{3}, a in Real, u in Real^{3}} |- (u otimes (a * v) otimes (w + x + y) otimes z) = ((a * (u otimes v otimes w otimes z)) + (a * (u otimes v otimes x otimes z)) + (a * (u otimes v otimes y otimes z)))

In [46]:
tensor_prod_06

(x + y) otimes z

In [47]:
tensor_prod_06.distribution(0)

{x in Real^{3}, y in Real^{3}, z in Real^{3}} |- ((x + y) otimes z) = ((x otimes z) + (y otimes z))

In [48]:
tensor_prod_07

x otimes (y + z)

In [49]:
tensor_prod_07.distribution(1)

{x in Real^{3}, y in Real^{3}, z in Real^{3}} |- (x otimes (y + z)) = ((x otimes y) + (x otimes z))

In [50]:
tensor_prod_08

u otimes v otimes (w + x + y) otimes z

In [51]:
tensor_prod_08.distribution(2)

{x in Real^{3}, y in Real^{3}, v in Real^{3}, w in Real^{3}, z in Real^{3}, u in Real^{3}} |- (u otimes v otimes (w + x + y) otimes z) = ((u otimes v otimes w otimes z) + (u otimes v otimes x otimes z) + (u otimes v otimes y otimes z))

In [52]:
tensor_prod_with_sum_01

x otimes (Sum_{i=1}^{3} f(i)) otimes z

In [53]:
tensor_prod_with_sum_01.distribution(1).rhs

Sum_{b_{1}=1}^{3} (x otimes f(b) otimes z)

In [54]:
tensor_prod_with_sum_02

(Sum_{i=2}^{4} f(i)) otimes z

In [55]:
tensor_prod_with_sum_02.distribution(0)

{z in Real^{3}, forall_{i in Natural} (f(i) in Real^{3})} |- ((Sum_{b_{1}=2}^{4} f(b_{1})) otimes z) = (Sum_{b_{1}=2}^{4} (f(b) otimes z))

In [56]:
tensor_prod_with_sum_03

x otimes (Sum_{i=2}^{4} f(i))

In [57]:
from proveit.linear_algebra.tensors import tensor_prod_distribution_over_summation
expr = tensor_prod_distribution_over_summation.instance_expr.instance_expr.instance_expr.instance_expr.rhs

expr: (a_{1} otimes ..a_{_a}.. otimes a_{i} otimes [Sum_{b_{1}, ..b_{_a}.., b_{j} | Q(b_{1}, ..b_{_a}.., b_{j})} f(b_{1}, ..b_{_a}.., b_{j})] otimes c_{1} otimes ..c_{_a}.. otimes c_{k}) \\  = [Sum_{b_{1}, ..b_{_a}.., b_{j} | Q(b_{1}, ..b_{_a}.., b_{j})} (a_{1} otimes ..a_{_a}.. otimes a_{i} otimes f(b) otimes c_{1} otimes ..c_{_a}.. otimes c_{k})]

In [58]:
tensor_prod_with_sum_03.distribution(1)

{x in Real^{3}, forall_{i in Natural} (f(i) in Real^{3})} |- (x otimes (Sum_{b_{1}=2}^{4} f(b_{1}))) = (Sum_{b_{1}=2}^{4} (x otimes f(b)))

In [59]:
tensor_prod_with_sum_04

u otimes v otimes w otimes x otimes (Sum_{i=1}^{5} f(i)) otimes z

In [60]:
tensor_prod_with_sum_04.distribution(4)

{x in Real^{3}, v in Real^{3}, w in Real^{3}, z in Real^{3}, forall_{i in Natural} (f(i) in Real^{3}), u in Real^{3}} |- (u otimes v otimes w otimes x otimes (Sum_{b_{1}=1}^{5} f(b_{1})) otimes z) = (Sum_{b_{1}=1}^{5} (u otimes v otimes w otimes x otimes f(b) otimes z))

In [61]:
# recall one of our TensorProd objects without a sum or summation:
tensor_prod_02

x otimes (a * y) otimes z

In [62]:
# we should get a meaningful error message when trying to distribute across
# a factor that is not a sum or summation:
try:
    tensor_prod_02.distribution(1)
    assert False, "Expecting a ValueError; should not get this far!"
except ValueError as the_error:
    print("ValueError: {}".format(the_error))

ValueError: Don't know how to distribute tensor product over <class 'proveit.linear_algebra.scalar_multiplication.scalar_mult.ScalarMult'> factor


### The `TensorProd.remove_vec_on_both_sides_of_equals()` and `TensorProd.insert_vec_on_both_sides_of_equals()` methods

In [63]:
help(TensorProd.remove_vec_on_both_sides_of_equals)

Help on function remove_vec_on_both_sides_of_equals in module proveit.linear_algebra.tensors.tensor_prod:

remove_vec_on_both_sides_of_equals(tensor_equality, idx, rhs_idx=None, *, field=None, **defaults_config)
    From an equality with tensor products of vectors on
    both sides, derive a similar equality but with the vector 
    operand removed at the particular given zero-based index (idx).
    A different index may be specified for the right side as the 
    left side by setting rhs_idx (i.e., if entries don't line up 
    due to differences of ExprRange entries), but the default will
    be to use the same.
    
    Keyword arguments are accepted for temporarily changing any
    of the attributes of proveit.defaults.



In [64]:
help(TensorProd.insert_vec_on_both_sides_of_equals)

Help on function insert_vec_on_both_sides_of_equals in module proveit.linear_algebra.tensors.tensor_prod:

insert_vec_on_both_sides_of_equals(tensor_equality, idx, vec, rhs_idx=None, *, field=None, **defaults_config)
    From an equality with tensor products of vectors on
    both sides, derive a similar equality but with a vector 
    operand inserted at the particular given zero-based index (idx).
    A different index may be specified for the right side as the 
    left side by setting rhs_idx (i.e., if entries don't line up 
    due to differences of ExprRange entries), but the default will
    be to use the same.
    
    Keyword arguments are accepted for temporarily changing any
    of the attributes of proveit.defaults.



In [65]:
tp_01, tp_02 = (TensorProd(x, y, z), TensorProd(x, u, z))

tp_01: x otimes y otimes z
tp_02: x otimes u otimes z

In [66]:
equality_01 = Equals(tp_01, tp_02)

equality_01: (x otimes y otimes z) = (x otimes u otimes z)

In [67]:
defaults.assumptions += (equality_01,)

We can access `TensorProd.remove_vec_on_both_sides_of_equals()` and `TensorProd.insert_vec_on_both_sides_of_equals()` via `Equals.remove_vec_on_both_sides` and `Equals.insert_vec_on_both_sides` respectively which are methods generated on-the-fly by finding methods with the `_on_both_sides_of_equals` suffix associated with the Equals expression.  The docstrings are set to be the same:

In [68]:
help(equality_01.remove_vec_on_both_sides)

Help on function remove_vec_on_both_sides_of_equals in module proveit.linear_algebra.tensors.tensor_prod:

remove_vec_on_both_sides_of_equals(tensor_equality, idx, rhs_idx=None, *, field=None, **defaults_config)
    From an equality with tensor products of vectors on
    both sides, derive a similar equality but with the vector 
    operand removed at the particular given zero-based index (idx).
    A different index may be specified for the right side as the 
    left side by setting rhs_idx (i.e., if entries don't line up 
    due to differences of ExprRange entries), but the default will
    be to use the same.
    
    Keyword arguments are accepted for temporarily changing any
    of the attributes of proveit.defaults.



In [69]:
equality_02 = equality_01.remove_vec_on_both_sides(0)

equality_02: {x in Real^{3}, (x otimes y otimes z) = (x otimes u otimes z), y in Real^{3}, z in Real^{3}, u in Real^{3}, x != 0(Real^{3})} |- (y otimes z) = (u otimes z)

In [70]:
equality_03 = equality_02.remove_vec_on_both_sides(1)

equality_03: {z != 0(Real^{3}), x in Real^{3}, (x otimes y otimes z) = (x otimes u otimes z), y in Real^{3}, z in Real^{3}, u in Real^{3}, x != 0(Real^{3})} |- y = u

The example above also demonstrates the auto-simplification of unary tensor products.

Now let's insert vectors into the tensor products.  Starting with the special case where we don't start out with a tensor product, we must call the `TensorProd.insert_vec_on_both_sides_of_equals` method directly: 

In [71]:
equality_04 = TensorProd.insert_vec_on_both_sides_of_equals(
    equality_03, 0, z)

equality_04: {z != 0(Real^{3}), x in Real^{3}, (x otimes y otimes z) = (x otimes u otimes z), y in Real^{3}, z in Real^{3}, u in Real^{3}, x != 0(Real^{3})} |- (z otimes y) = (z otimes u)

In the furture, we could have `CartExp` and other vector space expression classes implement `left_tensor_prod_both_sides_of_equals` (and `right_tensor_prod_both_sides_of_equals`) so then the above would be implemented via `equality.left_tensor_prod_both_sides(z)` insead.

In [72]:
#equality_04 = equality_03.left_tensor_prod_both_sides(z)

Now that we have tensor products on both sides, we can call `insert_vec_on_both_sides` in the equality.

In [73]:
equality_05 = equality_04.insert_vec_on_both_sides(2, x)

equality_05: {z != 0(Real^{3}), x in Real^{3}, (x otimes y otimes z) = (x otimes u otimes z), y in Real^{3}, z in Real^{3}, u in Real^{3}, x != 0(Real^{3})} |- (z otimes y otimes x) = (z otimes u otimes x)

### Related Testing: the `ExprTuple.range_expansion()` method

Operating on an ExprTuple whose single entry is an ExprRange that represents a finite list of elements, the `ExprTuple.range_expansion()` method converts self to an ExprTuple with a finite listing of explicit arguments.

For example, letting $\text{expr_tuple_01} = (x_1,\ldots,x_3)$ and then calling `expr_tuple_01.range_expansion()` deduces and returns

$\vdash ((x_1,\ldots,x_3) = (x_1, x_2, x_3))$

The reason for including this here in the demonstrations notebook for the linear_algebra package is that the method can be utilized to convert something like

$x_1 \otimes \ldots \otimes x_3$

to the more explicit

$x_1 \otimes x_2 \otimes x_3$

In [74]:
# create an example ExprRange
example_expr_range = ExprRange(i, IndexedVar(x, i), one, three)

example_expr_range: x_{1}, ..x_{i}.., x_{3}

In [75]:
# use the example ExprRange as the arg(s) for a TensorProd:
example_tensor_prod_over_range = TensorProd(example_expr_range)

example_tensor_prod_over_range: x_{1} otimes ..x_{i}.. otimes x_{3}

In [76]:
# find the ExprTuple equivalent to the (ExprTuple-wrapped) ExprRange:
expr_range_eq_expr_tuple = ExprTuple(example_expr_range).range_expansion()

expr_range_eq_expr_tuple: |- (x_{1}, ..x_{a}.., x_{3}) = (x_{1}, x_{2}, x_{3})

In [77]:
# find the equivalent explicit tensor prod
example_tensor_prod_over_range.inner_expr().operands.substitution(
         expr_range_eq_expr_tuple)

|- (x_{1} otimes ..x_{_a}.. otimes x_{3}) =  \\ (x_{1} otimes x_{2} otimes x_{3})

In [78]:
%end demonstrations